In [1]:
pip install llama-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.68.post1
    Uninstalling llama-index-core-0.10.68.post1:
      Successfully uninstalled llama-index-core-0.10.68.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-vector-stores-qdrant 0.1.4 requires llama-index-core<0.11.0,>=0.10.1, but you have llama-index-core 0.12.9 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.callbacks import CallbackManager

In [3]:
wiki_titles = [
    "Toronto",
    "Seattle"
]

In [4]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [5]:
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

In [6]:
city_docs

{'Toronto': [Document(id_='cfc53c6e-b6f5-4313-b4ab-0e66195f448d', embedding=None, metadata={'file_path': 'data/Toronto.txt', 'file_name': 'Toronto.txt', 'file_type': 'text/plain', 'file_size': 82590, 'creation_date': '2025-01-01', 'last_modified_date': '2025-01-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Toronto is the most populous city in Canada and the capital city of the Canadian province of Ontario. With a population of 2,794,356 in 2021, it is the fourth-most populous city in North America. The city is the anchor of the Golden Horseshoe, an urban agglomeration of 9,765,188 people (as of 2021) surrounding the western end of 

In [7]:
%pip install llama-index-llms-groq

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install llama_index.embeddings.ollama

Note: you may need to restart the kernel to use updated packages.


In [12]:
from llama_index.llms.groq import Groq
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding

In [13]:
Settings.llm = Groq(model="llama3-70b-8192", api_key="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")

In [14]:
Settings.embed_model = OllamaEmbedding(
    model_name="nomic-embed-text",
    embed_batch_size=10
)

In [15]:
from llama_index.core.agent import ReActAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os

In [16]:
node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

In [17]:
from llama_index.llms.ollama import Ollama

In [18]:
for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])
    all_nodes.extend(nodes)

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
        )

    # build summary index
    summary_index = SummaryIndex(nodes)
    # define query engines
    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {wiki_title}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    function_llm = Ollama(model="llama3.2", temperature=0.1)
    agent = ReActAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {wiki_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[wiki_title] = agent
    query_engines[wiki_title] = vector_index.as_query_engine(
        similarity_top_k=2
    )

In [19]:
agents

{'Toronto': <llama_index.core.agent.react.base.ReActAgent at 0x13b2e8830>,
 'Seattle': <llama_index.core.agent.react.base.ReActAgent at 0x13aaa8f50>}

In [20]:
query_engines

{'Toronto': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x13a4134d0>,
 'Seattle': <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine at 0x12fec7890>}

In [21]:
# define tool for each document agent
all_tools = []
for wiki_title in wiki_titles:
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        f" this tool if you want to answer any questions about {wiki_title}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[wiki_title],
        metadata=ToolMetadata(
            name=f"tool_{wiki_title}",
            description=wiki_summary,
        ),
    )
    all_tools.append(doc_tool)

In [22]:
all_tools

In [23]:
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [24]:
obj_index

In [25]:
from llama_index.core.agent import ReActAgent

top_agent = ReActAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about a set of given cities.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

In [26]:
top_agent

In [27]:
all_nodes

[TextNode(id_='77664a93-074d-46be-a7d1-a4c3e6d3a095', embedding=None, metadata={'file_path': 'data/Toronto.txt', 'file_name': 'Toronto.txt', 'file_type': 'text/plain', 'file_size': 82590, 'creation_date': '2025-01-01', 'last_modified_date': '2025-01-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cfc53c6e-b6f5-4313-b4ab-0e66195f448d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': 'data/Toronto.txt', 'file_name': 'Toronto.txt', 'file_type': 'text/plain', 'file_size': 82590, 'creation_date': '2025-01-01', 'last_modified_date': '2025-01-01'}, hash='a2255b0fd780f569cba907e65da1bdf357ad7d9805cdf7581e11ef8abbc67eaa'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='389e2527-7948-42c8-9

In [28]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

In [29]:
response = top_agent.query("Tell me about religions in Toronto")

> Running step 99efc978-56b1-4f95-9ae6-407b48d6ca29. Step input: Tell me about religions in Toronto
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: tool_Toronto
Action Input: {'input': 'Religion in Toronto'}
> Running step ec550399-be32-41ec-8509-d2617bb30c22. Step input: Religion in Toronto
Observation: Error: model requires more system memory (4.5 GiB) than is available (4.4 GiB)
> Running step 0e0b58d3-ef7e-46f0-8271-05ec7a5bf5a2. Step input: None
Thought: It seems like the tool_Toronto is not working properly due to memory issues. I'll try to use the tool again with a more specific input.
Action: tool_Toronto
Action Input: {'input': 'Religion in Toronto demographics'}
> Running step 91d0e7ab-20cb-48a9-adb3-07c5b8d49d6d. Step input: Religion in Toronto demographics
Observation: Error: model requires more system memory (4.5 GiB) than is available (4.4 GiB)
> Running step b10bea9f-6cf1-42d5-8aa1-f28f3927f3d1. Step inp